# web scraping

In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import pandas as pd
import numpy as np

base_url = 'https://www.mytek.tn/informatique/ordinateurs-portables/pc-gamer.html'
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')


html_string = str(soup)
html_lxml = etree.HTML(html_string)
nb_page = html_lxml.xpath('//*[@id="maincontent"]/div[4]/div[1]/div[2]/div[1]/ul/li[5]/a/span[2]')
nb_page=int(nb_page[0].text)

url_list = [base_url]
for number in range(2, nb_page+1):
    url = base_url + f'?p={number}'
    url_list.append(url)
data = {
    "DISPONIBILITÉ": [],
    "gtin": [],
    "Marque": [],
    "Gamme": [],
    "PC Gamer": [],
    "Système d'exploitation": [],
    "Taille de l'écran": [],
    "Ecran": [],
    "Ecran Tactile": [],
    "Résolution Ecran": [],
    "Processeur": [],
    "Type Processeur": [],
    "Référence Processeur": [],
    "Fréquence Processeur": [],
    "Mémoire Cache": [],
    "Mémoire": [],
    "Disque Dur": [],
    "Type Disque Dur": [],
    "Carte Graphique": [],
    "Chipset Graphique": [],
    "Clavier":[],
    "Connecteurs": [],
    "connecteur d'alimentation": [],
    "Couleur": [],
    "Garantie": [],
    "Prix": []
}

rows_list = []

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.find_all('a', class_='core')
    a_tags = soup.find_all('a', class_='product-item-link')
    href_values = [a_tag.get('href') for a_tag in a_tags]

    for href in href_values:
        if href is not None:  # Check if href is not None
            response = requests.get(href)
            soup1 = BeautifulSoup(response.text, 'html.parser')
            html_string = str(soup1)
            html_lxml = etree.HTML(html_string)
            attributes = {
                "DISPONIBILITÉ": 1,
                "gtin": 2,
                "Marque": 3,
                "Gamme": 4,
                "PC Gamer": 5,
                "Système d'exploitation": 6,
                "Taille de l'écran": 7,
                "Ecran":8,
                "Ecran Tactile": 9,
                "Résolution Ecran": 10,
                "Processeur": 11,
                "Type Processeur": 12,
                "Référence Processeur": 13,
                "Fréquence Processeur": 14,
                "Mémoire Cache": 15,
                "Mémoire": 16,
                "Disque Dur": 17,
                "Type Disque Dur": 18,
                "Carte Graphique": 19,
                "Chipset Graphique": 20,
                "Clavier":21,
                "Connecteurs": 22,
                "connecteur d'alimentation": 23,
                "Couleur": 24,
                "Garantie": 25,
                "Prix": 26  
            }

            new_row = {}
            for attribute, row_number in attributes.items():
                price = None  
                if attribute == "Prix":
                    a=soup1.find_all("span",class_="price")[-1].text
                    cleaned_price = a.replace('\u202f', ' ').replace('\xa0', ' ')
                    new_row[attribute] = cleaned_price
                else:
                    info = html_lxml.xpath(f'//*[@id="product-attribute-specs-table"]/tbody/tr[{row_number}]/td')
                    if info:
                        new_row[attribute] = info[0].text.strip()
                    else:
                        new_row[attribute] = np.nan
            rows_list.append(new_row)
        else:
            pass

df_custom = pd.DataFrame(rows_list)



df_custom

,DISPONIBILITÉ,gtin,Marque,Gamme,PC Gamer,Système d'exploitation,Taille de l'écran,Ecran,Ecran Tactile,Résolution Ecran,...,Disque Dur,Type Disque Dur,Carte Graphique,Chipset Graphique,Clavier,Connecteurs,connecteur d'alimentation,Couleur,Garantie,Prix
0,En stock,82K202B6FG,LENOVO,IDEAPAD GAMING 3,oui,FreeDos,15.6 Pouces,Full HD,Non,1920 x 1080 pixels,...,512Go SSD,SSD,Nvidia GeForce,GeForce RTX 2050,Rétroéclairé blanc,"2x USB 3.2 Gen 1, 1x USB-C 3.2 Gen 1, 1x HDMI ...",Noir,2 ans,NaN,"2 199,000 DT"
1,En stock,82XV00UGFG,LENOVO,LOQ 15IRH8,oui,FreeDos,15.6 Pouces,Full HD,Non,1920 x 1080 pixels,...,512Go SSD,SSD,Nvidia GeForce,GeForce RTX 2050,Rétroéclairé blanc,"1x USB 3.2 Gen 1, 2x USB 3.2 Gen 2, 1x USB-C 3...",Gris,2 ans,NaN,"2 299,000 DT"
2,En stock,82K2029EFG,LENOVO,IDEAPAD GAMING 3,oui,Windows 11 Famille,15.6 Pouces,Full HD,Non,1920 x 1080 pixels,...,512Go SSD,SSD,Nvidia GeForce,GeForce RTX 2050,Rétroéclairé blanc,"2x USB 3.2 Gen 1, 1x USB-C 3.2 Gen 1, 1x HDMI ...",Noir,1 an,NaN,"2 249,000 DT"
3,En stock,82K2029EFG-12G,LENOVO,IDEAPAD GAMING 3,oui,Windows 11 Famille,15.6 Pouces,Full HD,Non,1920 x 1080 pixels,...,512Go SSD,SSD,Nvidia GeForce,GeForce RTX 2050,Rétroéclairé blanc,"2x USB 3.2 Gen 1, 1x USB-C 3.2 Gen 1, 1x HDMI ...",Noir,1 an,NaN,"2 265,000 DT"
4,En stock,82XV00UHFG,LENOVO,LOQ 15IRH8,oui,Windows 11 Famille,15.6 Pouces,Full HD,Non,1920 x 1080 pixels,...,512Go SSD,SSD,Nvidia GeForce,GeForce RTX 2050,Rétroéclairé blanc,"1x USB 3.2 Gen 1, 2x USB 3.2 Gen 2, 1x USB-C 3...",Gris,2 ans,NaN,"2 249,000 DT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,En stock,886M4EA,HP,OMEN 17,oui,Windows 11 Famille,17.3 Pouces,QHD,Non,2560 x 1440 pixels,...,1 To SSD,SSD,Nvidia GeForce,GeForce RTX 4080,"Rétroéclairé RGB, de grande taille, 4 zones et...","3x USB Type-A, 1x Thunderbolt™ 4 avec USB Type...",Noir,1 an,NaN,"8 799,000 DT"
229,En stock,886M3EA,HP,OMEN 17,oui,Windows 11 Famille,17.3 Pouces,QHD,Non,2560 x 1440 pixels,...,1 To SSD,SSD,Nvidia GeForce,GeForce RTX 4090,"Rétroéclairé RGB, de grande taille, 4 zones et...","3x USB Type-A, 1x Thunderbolt™ 4 avec USB Type...",Noir,1 an,NaN,"9 399,000 DT"
230,En arrivage,886M2EA,HP,OMEN 17,oui,Windows 11 Famille,17.3 Pouces,QHD,Non,2560 x 1440 pixels,...,2 To,SSD,Nvidia GeForce,GeForce RTX 4090,"Rétroéclairé RGB, de grande taille, 4 zones et...","3x USB Type-A, 1x Thunderbolt™ 4 avec USB Type...",Noir,1 an,NaN,"10 399,000 DT"
231,En stock,ASUS,ROG ZEPHYRUS G16,oui,Windows 11 Famille,16 Pouces,2.5K,Non,2560 x 1440 pixels,Intel Core Ultra 9,...,SSD,Nvidia GeForce,GeForce RTX 4080,Chiclet rétroéclairé 1 zone RVB,"1x USB 3.2 Gen 2 Type-C, 2x USB 3.2 Gen 2 Type...",Gris,2 ans,NaN,NaN,"11 159,000 DT"


# Data cleaning

In [2]:
df_custom = df_custom.drop(columns=["Clavier","Connecteurs","connecteur d'alimentation","Couleur", "Garantie"])

In [5]:
df_custom["gtin"].value_counts()

gtin
82K202B6FG                      1
GIGABYTE-G5MF5-I7-32G           1
B12VFK-639XFR-24G               1
TUF707ZC4-HX016W                1
BU-GF63-12VF-413XFR-32G-W10P    1
                               ..
15A12VE-463XFR                  1
G15-5530-I5-3Y                  1
GF63-12UDX-213XFR               1
845M7EA-32G                     1
82WQ00A6FG                      1
Name: count, Length: 219, dtype: int64

In [4]:
df_custom = df_custom[~df_custom['gtin'].isin(['HP', 'LENOVO', 'ASUS'])]

In [6]:
df_custom["Marque"].value_counts()

Marque
MSI               64
ASUS              46
LENOVO            44
HP                26
GIGABYTE          20
DELL              17
KATANA 15 B12V     1
G5 KF              1
Name: count, dtype: int64

In [7]:
index = df_custom[df_custom['Marque'] == 'KATANA 15 B12V'].index[0]

rows_with_index = df_custom.loc[index]

rows_with_index

new_values = {
    'DISPONIBILITÉ': 'En stock',
    'gtin': 'B12VEK-640XFR',
    'Marque': 'MSI',
    'Gamme': 'KATANA 15 B12V',
    'PC Gamer': 'oui',
    'Système d\'exploitation': 'FreeDos',
    'Taille de l\'écran': '15.6 Pouces',
    'Ecran': 'Full HD',
    'Ecran Tactile': 'Non',
    'Résolution Ecran': '1920 x 1080 pixels',
    'Processeur': 'Intel Core i7',
    'Type Processeur': '10 cœurs',
    'Référence Processeur': 'Intel® Core™ i7-12650H',
    'Fréquence Processeur': 'up to 4,70 GHz Turbo max',
    'Mémoire Cache': '24 Mo',
    'Mémoire': '8 Go',
    'Disque Dur': '512Go SSD',
    'Type Disque Dur': 'SSD',
    'Carte Graphique': 'Nvidia GeForce',
    'Chipset Graphique': 'GeForce RTX 4050',
    'Prix': '4 159,000 DT'  
}
df_custom = df_custom.copy()


df_custom.loc[index] = new_values

rows_with_index = df_custom.loc[index]

In [8]:
rows_with_index

DISPONIBILITÉ                             En stock
gtin                                 B12VEK-640XFR
Marque                                         MSI
Gamme                               KATANA 15 B12V
PC Gamer                                       oui
Système d'exploitation                     FreeDos
Taille de l'écran                      15.6 Pouces
Ecran                                      Full HD
Ecran Tactile                                  Non
Résolution Ecran                1920 x 1080 pixels
Processeur                           Intel Core i7
Type Processeur                           10 cœurs
Référence Processeur        Intel® Core™ i7-12650H
Fréquence Processeur      up to 4,70 GHz Turbo max
Mémoire Cache                                24 Mo
Mémoire                                       8 Go
Disque Dur                               512Go SSD
Type Disque Dur                                SSD
Carte Graphique                     Nvidia GeForce
Chipset Graphique              

In [9]:
df_custom = df_custom[df_custom['Marque'] != 'G5 KF']

In [10]:
df_custom['Prix'] = df_custom['Prix'].str.replace(' DT', '')
df_custom['Prix'] = df_custom['Prix'].str.replace(',', '')

In [11]:
df_custom.to_csv('mylaptop3.csv', index=False)

# stocker le fichier mylaptop3.csv dans un compte de stockage

In [12]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.exceptions import ResourceExistsError

# Set your Azure Blob Storage connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=mytek;AccountKey=BitMjk7gZSif9NFDayd5MUfPvvD5A8FV9O3lXXpoF9wugEvPehZTO5ttA3u2juY2FTyxAUCQKCG8+AStBWEXrw==;EndpointSuffix=core.windows.net"

# Set the name of the container where you want to upload the file
container_name = "mytek"

# Set the local file path
local_file_path = r"C:\Users\elabo\mylaptop3.csv"

# Set the name for the blob (file) in the container
blob_name = "datacloud3.csv"

# Create a BlobServiceClient object using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a ContainerClient object that represents the container
container_client = blob_service_client.get_container_client(container_name)

# Upload the file to the container
try:
    with open(local_file_path, "rb") as data:
        container_client.upload_blob(name=blob_name, data=data, overwrite=True)
    print("Blob uploaded successfully.")
except ResourceExistsError:
    print("error")


Blob uploaded successfully.
